In [1]:
import datetime
import gc
import numpy as np
import os
import pandas as pd
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import skew, kurtosis
from sklearn.svm import SVC
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [2]:
# Analyze the data
import numpy as np
import pandas as pd

# load the dataframe
df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
#test = pd.read_csv('20-newsgroups-ciphertext-challenge/test.csv')

# dataframe for only difficulty=1
data1 = df.query('difficulty==1')
# how many elements do have diffuculty class
print(df['difficulty'].value_counts()) 
# count repeated chars in ciphertext class
alp = pd.Series(Counter(''.join(data1['ciphertext'])))  
print(pd.crosstab(df['difficulty'], df['target']))
alp.head(10)

2    10024
4     9970
1     9589
3     9469
Name: difficulty, dtype: int64
target       0    1    2    3    4    5    6    7    8    9    10   11   12  \
difficulty                                                                    
1           420  465  360  346  320  466  361  331  380  486  540  563  344   
2           455  528  652  391  315  550  310  405  387  301  580  651  396   
3           394  567  695  386  313  524  293  422  331  460  479  534  338   
4           465  366  940  382  326  653  272  437  386  413  437  622  429   

target       13   14   15   16   17   18   19  
difficulty                                     
1           482  576  765  471  834  675  404  
2           500  566  692  536  912  547  350  
3           593  470  513  506  714  553  384  
4           599  463  582  589  693  539  377  


c     53752
1    505426
|      7490
F      6134
a    121319
A     58440
O    134813
2      8502
0     75960
'     34490
dtype: int64

In [17]:
# Navier-Bayes modeli
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 1])

token_data = [tokenize(i) for i in X]
X = [trimm(i) for i in token_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=0)
# trigger timer
start_time = timeit.default_timer()

# create and train model
skf = StratifiedKFold(n_splits=5)
for train, test in skf.split(X, y):

nb = Pipeline([('vect', CountVectorizer(analyzer = 'word',lowercase = False,ngram_range=(1, 1))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print(accuracy_score(y_pred, y_test))
# calculate time interval
elapsed = timeit.default_timer() - start_time
print("Elapsed time", elapsed)

0.2716371220020855


In [25]:
# LogisticRegression classifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 1])

token_data = [tokenize(i) for i in X]
X = [trimm(i) for i in token_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=0)

lr = Pipeline([('vect', CountVectorizer(analyzer = 'word',lowercase = False,ngram_range=(1, 1))),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(solver='lbfgs', n_jobs=1, C=1e5, multi_class='auto')),
              ])
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.5041710114702815


/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [31]:
# SGDClassifier 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 3])

token_data = [tokenize(i) for i in X]
#X = [trimm(i) for i in token_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=0)

svm = Pipeline([('vect', CountVectorizer(analyzer = 'word',tokenizer = tokenize,
                                        lowercase = False,ngram_range=(1, 1))),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-5,
                                     random_state=42, max_iter=5, tol=None)),])
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print(accuracy_score(y_pred, y_test))

/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.5302398331595412


In [5]:
# SGDClassifier with high accuracy
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,:-1]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 3])

token_data = [tokenize(i) for i in X]
#X = [trimm(i) for i in token_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=0)

vectorizer = CountVectorizer(analyzer = 'char',tokenizer = tokenize,lowercase = False,ngram_range=(1, 6))

estimator = SGDClassifier(loss='hinge', max_iter=1000, random_state=0,tol=1e-3, n_jobs=-1)

model = Pipeline([('selector',FunctionTransformer(lambda x: x['ciphertext'], validate=False)),
                  ('vectorizer', vectorizer), 
                  ('tfidf', TfidfTransformer()),
                  ('estimator', estimator)])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

ValueError: Found input variables with inconsistent numbers of samples: [3, 9589]

In [1]:
# Logistic regression
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import pandas as pd
import re
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
#import lightgbm as lgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
from nltk.corpus import stopwords
from scipy.sparse import hstack, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

#X = X.str.lower()
X = X.astype(str)

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 3])

token_data = [tokenize(i) for i in X]
X = [trimm(i) for i in token_data]

tf = TfidfVectorizer(token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')

# build TFIDF Vectorizer
tokens= ((u'(?ui)\\b\\w*[a-z]+\\w*\\b')) # makes sure it matches a word but contains at least one letter

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    #stop_words = 'english',
    strip_accents='ascii',
    analyzer='word',
    token_pattern=tokens,
    ngram_range=(1,2),
    dtype=np.float32,
    max_features=7500
)

# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    #stop_words = 'english',
    strip_accents='ascii',
    analyzer='char',
    token_pattern=tokens,
    ngram_range=(2, 4),
    dtype=np.float32,
    max_features=12000
)

word_vectorizer.fit(X)
char_vectorizer.fit(X)

train_word_features = word_vectorizer.transform(X)
train_char_features = char_vectorizer.transform(X)

train_features = hstack([train_char_features,train_word_features])

%time
print("Modeling..")

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(train_features, y, test_size=0.33)

lr = LogisticRegression(solver="sag", max_iter=100,class_weight='balanced',C=2.65,penalty='l2')
lr.fit(train_features,y)
lr_pred=lr.predict(X_test_tfidf)

accuracy_tfidf =accuracy_score(y_test_tfidf,lr_pred)

print(accuracy_tfidf)
print("Auc Score: ",np.mean(cross_val_score(lr, train_features, y, cv=3,)))

print(classification_report(y_test_tfidf,lr_pred))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
Modeling..


/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.9169036334913112


/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/uzun/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Auc Score:  0.5028861672903329
              precision    recall  f1-score   support

           0       0.76      0.95      0.84       147
           1       0.93      0.95      0.94       150
           2       0.83      0.89      0.86       117
           3       0.85      0.80      0.82        99
           4       0.65      0.97      0.78       103
           5       0.93      0.87      0.90       163
           6       0.96      0.92      0.94       119
           7       0.88      0.88      0.88       112
           8       0.89      0.93      0.91       126
           9       0.99      0.94      0.96       142
          10       0.99      0.95      0.97       186
          11       1.00      0.90      0.94       200
          12       0.75      0.94      0.84       111
          13       0.99      0.94      0.96       160
          14       0.98      0.86      0.92       185
          15       0.99      0.94      0.96       244
          16       0.91      0.93      0.92       

In [25]:
# xgboost olan kod
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[:1000], y[:1000], test_size=0.25)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import nltk

tokens= ((u'(?ui)\\b\\w*[a-z]+\\w*\\b')) # makes sure it matches a word but contains at least one letter 

# BURAYA BAAAAAAAAAAAAAAAAAAAAAAAAK
#data_dmatrix = xgb.DMatrix(data=X,label=y)
#cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
#                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier

print("Pipelining..")

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True,strip_accents='ascii',analyzer='word',token_pattern=tokens,
                              ngram_range=(1,2),dtype=np.float32,max_features=7500)),
    ('svd',   TruncatedSVD(algorithm='randomized', n_components=500)),
    ('clf',   XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, 
                            colsample_bytree=0.7, subsample=0.8, eval_metric='merror')),
])

from sklearn.model_selection import GridSearchCV

parameters = {
    #'tfidf__max_df': (0.25, 0.5, 0.75),
    #'tfidf__min_df': (0.001, 0.0025, 0.005),
    #'tfidf__max_features': (50000, 100000, 150000),
    #'tfidf__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    #'svd__n_components': (250, 500, 750),
    #'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__max_iter': (10, 50, 80),
}

#gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
#gs_clf.fit(X_sample.message, y_sample)

#print("Best score: %0.3f" % gs_clf.best_score_)
#print("Best parameters set:")
#best_parameters = gs_clf.best_estimator_.get_params()
#for param_name in sorted(parameters.keys()):
#    print("\t%s: %r" % (param_name, best_parameters[param_name]))

text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_test)
print("The training predictions are ready")
print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions, average='weighted'))
print(classification_report(y_test, predictions))

'''
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD

vectorizer = TfidfVectorizer(tokenizer=Tokenizer, min_df=0.001, max_df=0.3)
X_tfidf = vectorizer.fit_transform(X)
#vectorizer.get_feature_names()

svd_transformer = TruncatedSVD(algorithm='randomized', n_components=300)
X_svd = svd_transformer.fit_transform(X_tfidf)

xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_svd, y)
print("The model is ready.")

X_test_tfidf = vectorizer.transform(X_test)
X_test_svd = svd_transformer.transform(X_test_tfidf)

xgb_predictions = xgb_classifier.predict(X_test_svd)
predictions = xgb_predictions

print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions, average='weighted'))
print(classification_report(y_test, predictions))

# feature visualize
#xgb.plot_importance(xg_reg)
#plt.rcParams['figure.figsize'] = [5, 5]
#plt.show()
'''

Pipelining..
The training predictions are ready
Accuracy: 0.104
Precision: 0.097928078817734
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.25      0.12      0.17         8
           2       0.67      0.20      0.31        20
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         6
           5       0.07      0.27      0.11        11
           6       0.00      0.00      0.00        13
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00        18
          11       0.09      0.21      0.12        14
          12       0.00      0.00      0.00         6
          13       0.07      0.12      0.09        16
          14       0.00      0.00      0.00        11
          15       0.21      0.19      0.2

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom nltk.tokenize import word_tokenize\nfrom sklearn.decomposition import TruncatedSVD\n\nvectorizer = TfidfVectorizer(tokenizer=Tokenizer, min_df=0.001, max_df=0.3)\nX_tfidf = vectorizer.fit_transform(X)\n#vectorizer.get_feature_names()\n\nsvd_transformer = TruncatedSVD(algorithm=\'randomized\', n_components=300)\nX_svd = svd_transformer.fit_transform(X_tfidf)\n\nxgb_classifier = XGBClassifier()\nxgb_classifier.fit(X_svd, y)\nprint("The model is ready.")\n\nX_test_tfidf = vectorizer.transform(X_test)\nX_test_svd = svd_transformer.transform(X_test_tfidf)\n\nxgb_predictions = xgb_classifier.predict(X_test_svd)\npredictions = xgb_predictions\n\nprint("Accuracy:", accuracy_score(y_test, predictions))\nprint("Precision:", precision_score(y_test, predictions, average=\'weighted\'))\nprint(classification_report(y_test, predictions))\n\n# feature visualize\n#xgb.plot_importance(xg_reg)\n#plt.rcParams[\'figure.figsize\'] = [5, 5]

In [25]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 3])

token_data = [tokenize(i) for i in X]
X = [trimm(i) for i in token_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

tokens= ((u'(?ui)\\b\\w*[a-z]+\\w*\\b')) # makes sure it matches a word but contains at least one letter 

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True,strip_accents='ascii',analyzer='word',token_pattern=tokens,
                              ngram_range=(1,2),dtype=np.float32,max_features=7500)),
    ('clf',   RandomForestClassifier(n_estimators=500,max_features='log2',min_samples_split=4)),
])
#fit_transform araştır!!!!!!!!!!!!!!!!

text_clf.fit(X_train,y_train)
predictions = text_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions, average='weighted'))
print(classification_report(y_test, predictions))

Accuracy: 0.4432860717264387
Precision: 0.4917226646914811
              precision    recall  f1-score   support

           0       0.43      0.20      0.27        99
           1       0.43      0.45      0.44       112
           2       0.62      0.39      0.48        90
           3       0.41      0.24      0.30        88
           4       0.44      0.13      0.21        82
           5       0.38      0.37      0.37       104
           6       0.52      0.49      0.51        91
           7       0.62      0.08      0.14        98
           8       0.59      0.39      0.47        84
           9       0.64      0.49      0.56       124
          10       0.50      0.63      0.56       144
          11       0.57      0.54      0.55       142
          12       0.74      0.15      0.25        91
          13       0.75      0.27      0.39       146
          14       0.55      0.62      0.58       141
          15       0.34      0.82      0.48       184
          16       0.5

In [11]:
# PCA 
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

df = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = df.query('difficulty==1')
X = data_1.iloc[:,-2]
y = data_1.iloc[:,-1]

def tokenize(text): 
    return text.split("1")

def trimm(text):
    return ' '.join([i for i in text if len(i) > 3])

token_data = [tokenize(i) for i in X]
X = [trimm(i) for i in token_data]

word2vec = Word2Vec(X, min_count=2)  
a=cosine_similarity(train_word_features, train_word_features)
#vocabulary = word2vec.wv.vocab
#print(a)

pca = PCA(n_components=10)

principalComponents = pca.fit_transform(a)

principalDf = pd.DataFrame(data = principalComponents)
print(principalDf.shape)
#finalDf = pd.concat([principalDf, y], axis = 1)
#finalDf

(9589, 10)


In [1]:
# Feature extraction
import datetime
import gc
import numpy as np
import os
import pandas as pd
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import skew, kurtosis
import lightgbm as lgb

import Levenshtein
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

from tqdm import tqdm

train = pd.read_csv('20-newsgroups-ciphertext-challenge/train.csv')
data_1 = train.query('difficulty==1')
X = data_1.iloc[:,:-1]
y = data_1.iloc[:,-1]

'''
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])
xtrain_count =  count_vect.transform(train_x)

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)

ensemble.RandomForestClassifier()

diff1['ciphertext'] = diff1['ciphertext'].apply(lambda x: x.replace('1', ' '))
diff2['ciphertext'] = diff2['ciphertext'].apply(lambda x: x.replace('8', ' '))
diff3['ciphertext'] = diff3['ciphertext'].apply(lambda x: x.replace('8', ' '))
diff4['ciphertext'] = diff4['ciphertext'].apply(lambda x: x.replace('8', ' '))
'''

def extract_features(df):
    #df['nunique'] = df['ciphertext'].apply(lambda x: len(np.unique(x)))
    #df['len'] = df['ciphertext'].apply(lambda x: len(x))

    def count_chars(x):
        n_l = 0 # count letters
        n_n = 0 # count numbers
        n_s = 0 # count symbols
        n_ul = 0 # count upper letters
        n_ll = 0 # count lower letters
        for i in range(0, len(x)):
            if x[i].isalpha():
                n_l += 1
                if x[i].isupper():
                    n_ul += 1
                elif x[i].islower():
                    n_ll += 1
            elif x[i].isdigit():
                n_n += 1
            else:
                n_s += 1

        return pd.Series([n_l, n_n, n_s, n_ul, n_ll])

    cols = ['n_l', 'n_n', 'n_s', 'n_ul', 'n_ll']
    for c in cols:
        df[c] = 0
    tqdm.pandas(desc='count_chars')
    df[cols] = df['ciphertext'].progress_apply(lambda x: count_chars(x))
    for c in cols:
        df[c] /= df['len']

    tqdm.pandas(desc='distances')
    df['Levenshtein_distance'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.distance(x, x[::-1]))
    df['Levenshtein_ratio'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.ratio(x, x[::-1]))
    df['Levenshtein_jaro'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.jaro(x, x[::-1]))
    df['Levenshtein_hamming'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.hamming(x, x[::-1]))

    for m in range(1, 5):
        df['Levenshtein_distance_m{}'.format(m)] = df['ciphertext'].progress_apply(lambda x: Levenshtein.distance(x[:-m], x[m:]))
        df['Levenshtein_ratio_m{}'.format(m)] = df['ciphertext'].progress_apply(lambda x: Levenshtein.ratio(x[:-m], x[m:]))
        df['Levenshtein_jaro_m{}'.format(m)] = df['ciphertext'].progress_apply(lambda x: Levenshtein.jaro(x[:-m], x[m:]))
        df['Levenshtein_hamming_m{}'.format(m)] = df['ciphertext'].progress_apply(lambda x: Levenshtein.hamming(x[:-m], x[m:]))
    
    df['Levenshtein_distance_h'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.distance(x[:len(x)//2], x[len(x)//2:]))
    df['Levenshtein_ratio_h'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.ratio(x[:len(x)//2], x[len(x)//2:]))
    df['Levenshtein_jaro_h'] = df['ciphertext'].progress_apply(lambda x: Levenshtein.jaro(x[:len(x)//2], x[len(x)//2:]))
    
    # All symbols stats
    def strstat(x):
        r = np.array([ord(c) for c in x])
        return pd.Series([
            np.sum(r), 
            np.mean(r), 
            np.std(r), 
            np.min(r), 
            np.max(r),
            skew(r), 
            kurtosis(r),
            ])
    cols = ['str_sum', 'str_mean', 'str_std', 'str_min', 'str_max', 'str_skew', 'str_kurtosis']
    for c in cols:
        df[c] = 0
    tqdm.pandas(desc='strstat')
    df[cols] = df['ciphertext'].progress_apply(lambda x: strstat(x))
    
    # Digit stats
    def str_digit_stat(x):
        r = np.array([ord(c) for c in x if c.isdigit()])
        if len(r) == 0:
            r = np.array([0])
        return pd.Series([
            np.sum(r), 
            np.mean(r), 
            np.std(r), 
            np.min(r), 
            np.max(r),
            skew(r), 
            kurtosis(r),
            ])
    cols = ['str_digit_sum', 'str_digit_mean', 'str_digit_std', 'str_digit_min', 
        'str_digit_max', 'str_digit_skew', 'str_digit_kurtosis']
    for c in cols:
        df[c] = 0
    tqdm.pandas(desc='str_digit_stat')
    df[cols] = df['ciphertext'].progress_apply(lambda x: str_digit_stat(x))

print('Extracting features for train:')
extract_features(train)
train.head()

Extracting features for train:


str_digit_stat: 100%|██████████| 39052/39052 [00:27<00:00, 1425.88it/s]


,Id,difficulty,ciphertext,target,nunique,len,n_l,n_n,n_s,n_ul,...,str_max,str_skew,str_kurtosis,str_digit_sum,str_digit_mean,str_digit_std,str_digit_min,str_digit_max,str_digit_skew,str_digit_kurtosis
0,ID_88b9bbd73,4,"ob|I K?zzhX*L{83B3Z,FuL*Pusm$83L\t@r$$*38,8s...",10,1,300,0.473333,0.170000,0.356667,0.246667,...,127.0,-0.091650,-0.950683,2664.0,52.235294,3.299366,48.0,57.0,-0.001394,-1.549887
1,ID_f489bd59f,1,c1|FaAO120O'8ovfoy1W#atvGs1[1s1[1/1]O-a8o1-...,13,1,300,0.383333,0.240000,0.376667,0.103333,...,124.0,-0.047954,-1.144330,3568.0,49.555556,2.146631,48.0,56.0,2.564841,4.938975
2,ID_f90fee9c7,2,1*e4N8$f$0ccOuihkHek$k*V*hoeV$Hj8VhH8...,19,1,300,0.430000,0.146667,0.423333,0.176667,...,127.0,-0.063138,-1.231967,2406.0,54.681818,2.274091,48.0,57.0,-1.587334,1.299930
3,ID_8303ced65,1,O8v^10O#to1'#^'^tv1^]s111t01Otaq>-ata_1...,17,1,300,0.420000,0.246667,0.333333,0.153333,...,125.0,-0.077411,-0.983794,3682.0,49.756757,2.276857,48.0,56.0,2.245122,3.317398
4,ID_72abc2cb7,2,eV}H}khfe4b8'S.Vc}{A .#VikV.fV?{$f7$Hjb8...,0,1,300,0.433333,0.113333,0.453333,0.133333,...,126.0,-0.241807,-1.200268,1875.0,55.147059,1.477923,51.0,56.0,-1.675256,1.324605
